In [ ]:
import pandas as pd
import osmnx as ox
import networkx as nx
import random
import numpy as np
import warnings
import json
from datetime import datetime, timedelta
import geopandas as gpd
from tqdm.notebook import tqdm
import subprocess
import sys
import xml.etree.ElementTree as ET
from xml.dom import minidom

# --- Pengaturan Awal ---
warnings.filterwarnings('ignore', 'GeoSeries.notna', UserWarning)
ox.settings.use_cache = True
ox.settings.all_oneway = True
# Gunakan area yang lebih kecil untuk memastikan proses cepat dan berhasil
place_name = "Jakarta Pusat, Jakarta, Indonesia" 
jumlah_agen = 200

# ==============================================================================
# LANGKAH 1: BUAT FILE JARINGAN .net.xml YANG DIJAMIN KONSISTEN
# ==============================================================================
print(f"--- LANGKAH 1: MEMBUAT JARINGAN SUMO UNTUK '{place_name}' ---")
osm_filename = "network.osm.xml"
net_filename = "network.net.xml"

# 1a. Unduh data OSM mentah (unsimplified)
print("Mengunduh data mentah OSM...")
G_unsimplified = ox.graph_from_place(place_name, network_type='all', simplify=False)
ox.save_graph_xml(G_unsimplified, filepath=osm_filename)
print(f"Data mentah OSM disimpan ke '{osm_filename}'")

# 1b. Jalankan netconvert dari dalam Python
print("Menjalankan netconvert untuk membuat file jaringan...")
try:
    subprocess.run([
        'netconvert',
        '--osm-files', osm_filename,
        '-o', net_filename,
        '--proj.utm',
        '--lefthand',
        '--verbose'
    ], check=True, capture_output=True, text=True) # capture_output untuk menyembunyikan log panjang
    print(f"File jaringan '{net_filename}' berhasil dibuat.")
except FileNotFoundError:
    print("\nERROR: Perintah 'netconvert' tidak ditemukan.")
    print("Pastikan SUMO sudah terinstal dan path 'sumo/bin' ada di environment variable sistem Anda.")
    sys.exit()
except subprocess.CalledProcessError as e:
    print(f"\nERROR: netconvert gagal dijalankan. Pesan error:\n{e.stderr}")
    sys.exit()


In [ ]:
# ==============================================================================
# LANGKAH 2: BUAT DATA AGEN MENGGUNAKAN GRAF YANG SESUAI
# ==============================================================================
print(f"\n--- LANGKAH 2: MEMBUAT DATA AGEN ---")

# 2a. Muat graf yang disederhanakan dan diproyeksikan untuk perhitungan Python
print("Memuat dan memproyeksikan graf untuk analisis...")
G_simplified_unprojected = ox.graph_from_place(place_name, network_type='all', simplify=True)
G_projected = ox.project_graph(G_simplified_unprojected)
target_crs = G_projected.graph['crs']
print("Graf untuk analisis berhasil dimuat.")

# 2b. Muat dan proyeksikan data POI
print("Memuat dan memproyeksikan data POI...")
gdf_rumah = ox.features_from_place(place_name, {"building": ["apartments", "residential", "house"]}).to_crs(target_crs)
gdf_kerja = ox.features_from_place(place_name, {"amenity": ["office", "bank"], "landuse": "commercial"}).to_crs(target_crs)
gdf_kecamatan = ox.features_from_place(place_name, {"admin_level": "7"})
gdf_kecamatan = gdf_kecamatan[gdf_kecamatan.geom_type.isin(['Polygon', 'MultiPolygon'])].to_crs(target_crs)
print("Data POI berhasil dimuat.")

# 2c. Fungsi dan logika pembuatan agen
def get_random_location(gdf, place_type, default_name="Lokasi"):
    if gdf.empty: return None
    random_place = gdf.sample(1).iloc[0]
    point = random_place.geometry.centroid
    name = random_place.get('name', default_name)
    if pd.isna(name): name = default_name
    return {"nama_tempat": str(name), "jenis": place_type, "koordinat": {"x": point.x, "y": point.y}}

print("Membuat sekuens harian untuk agen...")
data_agen = []
for i in tqdm(range(1, jumlah_agen + 1), desc="Membuat Agen"):
    agent_id = f"agent_{i}"
    sequence = []
    kecamatan_rumah = gdf_kecamatan.sample(1)
    area_rumah = kecamatan_rumah.geometry.iloc[0]
    nama_kecamatan_rumah = kecamatan_rumah['name'].iloc[0]
    rumah_lokal = gdf_rumah[gdf_rumah.within(area_rumah)]
    rumah_dict = get_random_location(rumah_lokal if not rumah_lokal.empty else gdf_rumah, "rumah", f"Rumah di {nama_kecamatan_rumah}")
    kantor_dict = get_random_location(gdf_kerja, "kerja", "Kantor")
    if rumah_dict is None or kantor_dict is None: continue
    sequence.extend([rumah_dict, kantor_dict, rumah_dict])
    data_agen.append({"agent_id": agent_id, "sequence_harian": sequence})


In [ ]:
# ==============================================================================
# LANGKAH 3: BUAT DATA PERJALANAN (TIMESTAMP) DENGAN VALIDASI EDGE ID
# ==============================================================================
print(f"\n--- LANGKAH 3: MEMBUAT JADWAL PERJALANAN DENGAN EDGE ID ---")

DURASI = {"kerja": (8.0, 9.5)}
def get_trip_info(graph, x, y):
    try:
        edge = ox.nearest_edges(graph, x, y)
        edge_id = graph.edges[edge]['osmid']
        if isinstance(edge_id, list): edge_id = edge_id[0]
        return str(edge_id)
    except:
        return None

data_perjalanan_final = []
agen_valid_count = 0
for agen in tqdm(data_agen, desc="Memvalidasi & Memproses Agen"):
    agent_id = agen['agent_id']; sequence = agen['sequence_harian']
    waktu_sekarang = datetime.now().replace(hour=random.randint(6, 8), minute=random.randint(0, 59), second=0, microsecond=0)
    perjalanan_agen_saat_ini = []; seluruh_sequence_valid = True
    for i in range(len(sequence) - 1):
        lokasi_awal = sequence[i]; lokasi_tujuan = sequence[i+1]
        coord_awal = lokasi_awal['koordinat']; coord_tujuan = lokasi_tujuan['koordinat']
        
        edge_awal = get_trip_info(G_projected, coord_awal['x'], coord_awal['y'])
        edge_akhir = get_trip_info(G_projected, coord_tujuan['x'], coord_tujuan['y'])

        if not all([edge_awal, edge_akhir]):
            seluruh_sequence_valid = False; break
        
        perjalanan_agen_saat_ini.append([agent_id, waktu_sekarang, edge_awal, edge_akhir])
        
        jenis_aktivitas = lokasi_tujuan['jenis']
        durasi_min, durasi_max = DURASI.get(jenis_aktivitas, (0,0))
        # Perkiraan waktu tiba untuk jadwal berikutnya (bisa disempurnakan dengan perhitungan jarak)
        waktu_sekarang += timedelta(minutes=random.randint(20, 60)) # Asumsi waktu perjalanan
        waktu_sekarang += timedelta(hours=random.uniform(durasi_min, durasi_max))
        
    if seluruh_sequence_valid:
        data_perjalanan_final.extend(perjalanan_agen_saat_ini)
        agen_valid_count += 1

--- LANGKAH 1: MEMBUAT JARINGAN SUMO UNTUK 'Menteng, Jakarta, Indonesia' ---
Mengunduh data mentah OSM...
Data mentah OSM disimpan ke 'network.osm.xml'
Menjalankan netconvert untuk membuat file jaringan...
File jaringan 'network.net.xml' berhasil dibuat.

--- LANGKAH 2: MEMBUAT DATA AGEN ---
Memuat dan memproyeksikan graf untuk analisis...
Graf untuk analisis berhasil dimuat.
Memuat dan memproyeksikan data POI...
Data POI berhasil dimuat.
Membuat sekuens harian untuk agen...


Membuat Agen:   0%|          | 0/200 [00:00<?, ?it/s]


--- LANGKAH 3: MEMBUAT JADWAL PERJALANAN DENGAN EDGE ID ---


Memvalidasi & Memproses Agen:   0%|          | 0/200 [00:00<?, ?it/s]

In [16]:
# ==============================================================================
# LANGKAH 4 & 5: UBAH KE .TRIPS.XML DAN JALANKAN DUAROUTER
# ==============================================================================
print(f"\n--- LANGKAH 4: MENGONVERSI {agen_valid_count} AGEN VALID KE FILE TRIPS ---")
df_perjalanan = pd.DataFrame(data_perjalanan_final, columns=["agent_id", "jam_berangkat", "edge_awal", "edge_akhir"])
trips_filename = "agent.trips.xml"

root = ET.Element('trips')
ET.SubElement(root, 'vType', id="mobil_pribadi", accel="2.6", decel="4.5", length="5", maxSpeed="70", color="1,0,0")
for index, row in df_perjalanan.iterrows():
    detik_berangkat = (row['jam_berangkat'] - row['jam_berangkat'].replace(hour=0, minute=0, second=0)).total_seconds()
    ET.SubElement(root, 'trip', {
        'id': f"{row['agent_id']}_{index}", 'type': 'mobil_pribadi', 'depart': str(int(detik_berangkat)),
        'from': row['edge_awal'], 'to': row['edge_akhir']
    })

xml_string = ET.tostring(root, 'utf-8')
reparsed = minidom.parseString(xml_string)
with open(trips_filename, "w", encoding="utf-8") as f:
    f.write(reparsed.toprettyxml(indent="    "))
print(f"File '{trips_filename}' berhasil dibuat.")

print(f"\n--- LANGKAH 5: MENJALANKAN DUAROUTER ---")
rou_filename = "agent_routes.rou.xml"
try:
    subprocess.run([
        'duarouter',
        '-n', net_filename,
        '--trip-files', trips_filename,
        '-o', rou_filename,
        '--ignore-errors',
        '--verbose'
    ], check=True, capture_output=True, text=True)
    print(f"File rute final '{rou_filename}' berhasil dibuat.")
    print("\n==============================================================================")
    print("SELURUH PROSES SELESAI! ANDA SIAP UNTUK SIMULASI.")
    print(f"File jaringan Anda adalah: '{net_filename}'")
    print(f"File rute Anda adalah: '{rou_filename}'")
    print("==============================================================================")
except FileNotFoundError:
    print("\nERROR: Perintah 'duarouter' tidak ditemukan.")
    print("Pastikan SUMO sudah terinstal dan path 'sumo/bin' ada di environment variable sistem Anda.")
    sys.exit()
except subprocess.CalledProcessError as e:
    print(f"\nERROR: duarouter gagal dijalankan. Pesan error:\n{e.stderr}")
    sys.exit()


--- LANGKAH 4: MENGONVERSI 200 AGEN VALID KE FILE TRIPS ---
File 'agent.trips.xml' berhasil dibuat.

--- LANGKAH 5: MENJALANKAN DUAROUTER ---
File rute final 'agent_routes.rou.xml' berhasil dibuat.

SELURUH PROSES SELESAI! ANDA SIAP UNTUK SIMULASI.
File jaringan Anda adalah: 'network.net.xml'
File rute Anda adalah: 'agent_routes.rou.xml'
